In [2]:
import csv
import ast

In [39]:
record=[]
count=0
#with open("new test result/baselineBERT/cpu/baselinebert_cpu_epoch_5_test_res.csv",newline='') as csvfile:
with open("new data/MMPBERT/mixprompt_all_epoch_20_test_res.csv",newline='') as csvfile:
    read=csv.reader(csvfile)
    for item in read:
        if(count==0):
            count+=1
            continue
        #tmp=item[:]
        tmp=item[:1]+item[13:]
        tmp[0]=int(tmp[0])
        tmp[1]=int(tmp[1])
        tmp[2]=int(tmp[2])
        
        #numpy version
        """tmp[3]=tmp[3][1:-1].split(' ')
        #print(tmp[3])
        n=len(tmp[3])
        logit=[]
        for i in range(n):
            if(tmp[3][i]==''):
                continue
            logit.append(float(tmp[3][i]))"""
        
        #normal version
        tmp[3]=ast.literal_eval(tmp[3])
        
        """tmp[3]=logit"""
        record.append(tmp)

print(record[0])
print(len(record))

[0, 2, 2, [-8.72199535369873, -8.812382698059082, -1.3904452323913574, -7.842248916625977, -14.883234977722168, -14.708634376525879, -14.298813819885254, -8.64927864074707]]
1148


In [40]:
#--------------------------recall rate-------------------------------

n=len(record)
cls_num=len(record[0][3])#class number used to define range of k
deno=[0 for i in range(cls_num)]#number of samples of each class
nume=[[0 for j in range(cls_num)] for i in range(cls_num)]#external range is for each class, internal range is for @k
#e.g. nume[i][j] is the numerator for calculating class i's @j recall rate

for i in range(0,n):
    label_i=record[i][1]
    
    #create a list containing tuples in form of (logit, index)
    ind_len=len(record[i][3])
    logit_index_list=zip(record[i][3], range(0,ind_len))
    logit_index_list=list(logit_index_list)
    logit_index_list=sorted(logit_index_list, key=lambda x : x[0], reverse=True)
    #logit_index_list=logit_index_list.map(lambda x:x[1])
    
    #turn the list above to a index list
    index_list=[]
    for item in logit_index_list:
        index_list.append(item[1])
    #print(logit_index_list)
    #print(index_list, record[i][1])
    #break
    
    #record info for calculating the @k recall rate
    deno[label_i]+=1
    for j in range(cls_num):
        #(j+1) is k
        if(label_i in index_list[:j+1]):
            nume[label_i][j]+=1

recall_rate_k=[0 for i in range(cls_num)]
#calculate the @k recall rate
for i in range(cls_num):
    #(i+1) is k
    tmp=cls_num
    for j in range(cls_num):
        #j is class
        if(deno[j]==0):
            tmp-=1
            continue
        recall_rate_k[i]+=((nume[j][i]/deno[j])*(deno[j]/sum(deno)))
        #recall_rate_k[i]+=(nume[j][i]/deno[j])
    #recall_rate_k[i]/=tmp

In [41]:
print(recall_rate_k)

[0.6027874564459931, 0.8318815331010453, 0.9207317073170732, 0.960801393728223, 0.9782229965156795, 0.9886759581881533, 0.9930313588850175, 1.0000000000000002]


In [42]:
#print(recall_rate_k)
for item in recall_rate_k:
    print(item)

0.6027874564459931
0.8318815331010453
0.9207317073170732
0.960801393728223
0.9782229965156795
0.9886759581881533
0.9930313588850175
1.0000000000000002


In [43]:
"""for i in range(cls_num):
    print(nume[i])
    print(deno[i])"""

'for i in range(cls_num):\n    print(nume[i])\n    print(deno[i])'

In [44]:
#--------------------------precision rate-------------------------------

n=len(record)
cls_num=len(record[0][3])#class number used to define range of k
deno=[0 for i in range(cls_num)]#number of samples of each class
nume=[[0 for j in range(cls_num)] for i in range(cls_num)]#external range is for each class, internal range is for @k
#e.g. nume[i][j] is the numerator for calculating class i's @j recall rate

for i in range(0,n):
    label_i=record[i][1]
    
    #create a list containing tuples in form of (logit, index)
    ind_len=len(record[i][3])
    logit_index_list=zip(record[i][3], range(0,ind_len))
    logit_index_list=list(logit_index_list)
    logit_index_list=sorted(logit_index_list, key=lambda x : x[0], reverse=True)
    #logit_index_list=logit_index_list.map(lambda x:x[1])
    
    #turn the list above to a index list
    index_list=[]
    for item in logit_index_list:
        index_list.append(item[1])
    #print(logit_index_list)
    #print(index_list, record[i][1])
    #break
    
    #record info for calculating the @k recall rate
    deno[label_i]+=1
    for j in range(cls_num):
        #(j+1) is k
        if(label_i in index_list[:j+1]):
            nume[label_i][j]+=1

precision_rate_k=[0 for i in range(cls_num)]
#calculate the @k recall rate
for i in range(cls_num):
    #(i+1) is k
    tmp=cls_num
    for j in range(cls_num):
        #j is class
        if(deno[j]==0):
            tmp-=1
            continue
        precision_rate_k[i]+=((nume[j][i]/((i+1)*deno[j]))*(deno[j]/sum(deno)))

In [45]:
#print(precision_rate_k)
for item in precision_rate_k:
    print(item)

0.6027874564459931
0.41594076655052264
0.30691056910569103
0.24020034843205576
0.1956445993031359
0.1647793263646922
0.14186162269785962
0.12500000000000003


In [46]:
f1_score_k=[]
for i in range(len(recall_rate_k)):
    tmp=2*(recall_rate_k[i]*precision_rate_k[i])/(recall_rate_k[i]+precision_rate_k[i])
    f1_score_k.append(tmp)

In [47]:
#print(f1_score_k)
for item in f1_score_k:
    print(item)

0.6027874564459931
0.5545876887340301
0.46036585365853655
0.38432055749128924
0.3260743321718932
0.2824788451966152
0.2482578397212544
0.22222222222222227


In [237]:
"""x=2*(0.9626168224299065*0.3208722741433022)/(0.9626168224299065+0.3208722741433022)
print(x)"""

'x=2*(0.9626168224299065*0.3208722741433022)/(0.9626168224299065+0.3208722741433022)\nprint(x)'